In [3]:
import pandas as pd
import librosa
import os
import numpy as np

In [3]:
clean_df = pd.read_csv('clean.csv')
#remove first 7 characters from utt_id and path
clean_df['utt_id'] = clean_df['utt_id'].str[7:]
clean_df['path'] = clean_df['path'].str[7:]
#os join path
clean_df['path'] = clean_df['path'].apply(lambda x: os.path.join('VAD_OAD2204', x))
#sort by utt_id
clean_df = clean_df.sort_values(by='utt_id')
clean_df['duration'] = 0
clean_df['label'] = 'speech'
print(len(clean_df))
# if start_s = 0, print it
# clean_df[clean_df['start_s'] == 0]

15496


In [ ]:
for i in range(0, len(clean_df)):
    y, sr = librosa.load(clean_df.iloc[i]['path'])
    clean_df['duration'].iloc[i] = librosa.get_duration(y=y, sr=sr)
clean_df.head()

# y, sr = librosa.load(clean_df['path'].iloc[0], 16000)
# print(librosa.get_duration(y, sr))
#new field duration fill with 0


In [49]:
total_clean = pd.read_csv('total_clean.csv')
total_clean.head()

,Unnamed: 0,utt_id,path,text,text_enhanced,spkid,start_s,end_s,problems,duration,label
0,1767,A1048013552#400#9005547914635336,/content/drive/Shareddrives/vad/NeMo_data/OAD2...,TẮT NHẠC,TẮT NHẠC,110622,1.318762,1.720913,NaN,3.845125,speech
1,1831,A1048013552#400#9005547919649544,/content/drive/Shareddrives/vad/NeMo_data/OAD2...,MỞ BÀI CHÚC BÉ NGỦ NGON,MỞ BÀI CHÚC BÉ NGỦ NGON,110622,1.372884,3.745972,NaN,4.711875,speech
2,1768,A1048013552#400#9005547925994899,/content/drive/Shareddrives/vad/NeMo_data/OAD2...,MỞ BÀI TRÁI TIM,MỞ BÀI TRAI TIM,110622,1.305186,2.872490,NaN,4.003750,speech
3,1751,A1048013552#400#9005548193534208,/content/drive/Shareddrives/vad/NeMo_data/OAD2...,MỞ BÀI BỐ ƠI BỐ ƠI BỐ BỐ ƠI BỐ ƠI,MỞ BÀI BỐ ƠI BỐ ƠI BỐ BỐ ƠI BỐ ƠI,110622,2.533318,5.159882,NaN,6.022500,speech
4,1746,A1048013552#400#9005548193562623,/content/drive/Shareddrives/vad/NeMo_data/OAD2...,MỞ BÀI BỐ ƠI BỐ ƠI BỐ ƠI,MỞ BÀI BỐ ƠI BỐ ƠI BỐ ƠI,110622,2.426467,4.852765,NaN,6.114375,speech


In [50]:
train_clean = total_clean[:12496]
test_clean = total_clean[12496:]

# train_df = train_clean[['utt_id', 'path']]
# train_df.to_csv('data_train/wav.scp', sep=' ', index=False, header=None)
# test_df = test_clean[['utt_id', 'path']]
# test_df.to_csv('data_test/wav.scp', sep=' ', index=False, header=None)

#print total of duration
print(sum(train_clean['duration'])/3600)
print(sum(test_clean['duration'])/3600)

14.206350746527807
3.342288888888892


In [25]:
def add_nonspeech(dataframe):
    segments_df = dataframe[['utt_id', 'start_s', 'end_s', 'duration','label']]
    for i in range(0, len(segments_df)):
        if segments_df['start_s'].iloc[i] != 0:
            #append new row with start_s = 0 and end_s = segments_df['start_s'].iloc[i], label = 'nonspeech'
            new_row = {'utt_id': segments_df['utt_id'].iloc[i], 'start_s': 0, 'end_s': segments_df['start_s'].iloc[i], 'duration': segments_df['duration'].iloc[i], 'label': 'nonspeech'}
            segments_df = segments_df.append(new_row, ignore_index=True)
        if segments_df['end_s'].iloc[i] != segments_df['duration'].iloc[i]:
            #append new row with start_s = segments_df['end_s'].iloc[i] and end_s = segments_df['duration'].iloc[i]
            new_row = {'utt_id': segments_df['utt_id'].iloc[i], 'start_s': segments_df['end_s'].iloc[i], 'end_s': segments_df['duration'].iloc[i], 'duration': segments_df['duration'].iloc[i], 'label': 'nonspeech'}
            segments_df = segments_df.append(new_row, ignore_index=True)
    #sort by utt_id, start_s, end_s
    segments_df = segments_df.sort_values(by=['utt_id', 'start_s'])
    #if it has same utt_id, new column 'segment_id' start from 1,2,3...
    segments_df['segment_id'] = segments_df.groupby('utt_id').cumcount()
    #add new column 'utt_id_segment_id' at first column and fill with utt_id + '_' + segment_id
    segments_df['utt_id_segment_id'] = segments_df['utt_id'].map(str) + '_' + segments_df['segment_id'].map(str)
    #move utt_id_segment_id to first column
    segments_df = segments_df[['utt_id_segment_id', 'utt_id', 'start_s', 'end_s', 'duration', 'label']]

    segments_df[['utt_id_segment_id', 'utt_id', 'start_s', 'end_s']].to_csv('data_train/segments', index = False, header = None, sep =' ')
    segments_df[['utt_id_segment_id', 'label']].to_csv('data_train/utt2spk', index=False, sep=' ', header=None)

In [26]:
def add_nonspeech2(dataframe):
    segments_df = dataframe[['utt_id', 'start_s', 'end_s', 'duration','label']]
    for i in range(0, len(segments_df)):
        if segments_df['start_s'].iloc[i] != 0:
            #append new row with start_s = 0 and end_s = segments_df['start_s'].iloc[i], label = 'nonspeech'
            new_row = {'utt_id': segments_df['utt_id'].iloc[i], 'start_s': 0, 'end_s': segments_df['start_s'].iloc[i], 'duration': segments_df['duration'].iloc[i], 'label': 'nonspeech'}
            segments_df = segments_df.append(new_row, ignore_index=True)
        if segments_df['end_s'].iloc[i] != segments_df['duration'].iloc[i]:
            #append new row with start_s = segments_df['end_s'].iloc[i] and end_s = segments_df['duration'].iloc[i]
            new_row = {'utt_id': segments_df['utt_id'].iloc[i], 'start_s': segments_df['end_s'].iloc[i], 'end_s': segments_df['duration'].iloc[i], 'duration': segments_df['duration'].iloc[i], 'label': 'nonspeech'}
            segments_df = segments_df.append(new_row, ignore_index=True)
    #sort by utt_id, start_s, end_s
    segments_df = segments_df.sort_values(by=['utt_id', 'start_s'])
    #if it has same utt_id, new column 'segment_id' start from 1,2,3...
    segments_df['segment_id'] = segments_df.groupby('utt_id').cumcount()
    #add new column 'utt_id_segment_id' at first column and fill with utt_id + '_' + segment_id
    segments_df['utt_id_segment_id'] = segments_df['utt_id'].map(str) + '_' + segments_df['segment_id'].map(str)
    #move utt_id_segment_id to first column
    segments_df = segments_df[['utt_id_segment_id', 'utt_id', 'start_s', 'end_s', 'duration', 'label']]

    segments_df[['utt_id_segment_id', 'utt_id', 'start_s', 'end_s']].to_csv('data_test/segments', index=False, header=None, sep=' ')
    segments_df[['utt_id_segment_id', 'label']].to_csv('data_test/utt2spk', index=False, sep=' ', header=None)

In [ ]:
add_nonspeech(train_clean)
add_nonspeech2(test_clean)

In [55]:
segments_df = test_clean[['utt_id', 'start_s', 'end_s']]
#add new column 'utt_id_segment_id' at first column and fill with utt_id + '_' + segment_id
segments_df['segment_id'] = segments_df.groupby('utt_id').cumcount()
segments_df['utt_id_segment_id'] = segments_df['utt_id'].map(str) + '_' + segments_df['segment_id'].map(str)
segments_df[['utt_id_segment_id', 'utt_id', 'start_s', 'end_s']].to_csv('data_test/segments_gr', index=False, header=None, sep=' ')


C:\Users\admin\AppData\Local\Temp\ipykernel_8728\1483742813.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segments_df['segment_id'] = segments_df.groupby('utt_id').cumcount()
C:\Users\admin\AppData\Local\Temp\ipykernel_8728\1483742813.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segments_df['utt_id_segment_id'] = segments_df['utt_id'].map(str) + '_' + segments_df['segment_id'].map(str)


In [47]:
data = pd.read_csv('data_train/segments', sep=' ', header=None)
data.columns = ['utt_id_segment_id', 'utt_id', 'start_s', 'end_s']
data.head()
#print row if start_s or end_s is NaN
data[data.isnull().any(axis=1)]

,utt_id_segment_id,utt_id,start_s,end_s


In [ ]:
# segments_df = total_clean[['utt_id', 'start_s', 'end_s', 'duration','label']]
# for i in range(0, len(segments_df)):
#     if segments_df['start_s'].iloc[i] != 0:
#         #append new row with start_s = 0 and end_s = segments_df['start_s'].iloc[i], label = 'nonspeech'
#         new_row = {'utt_id': segments_df['utt_id'].iloc[i], 'start_s': 0, 'end_s': segments_df['start_s'].iloc[i], 'duration': segments_df['duration'].iloc[i], 'label': 'nonspeech'}
#         segments_df = segments_df.append(new_row, ignore_index=True)
#     if segments_df['end_s'].iloc[i] != segments_df['duration'].iloc[i]:
#         #append new row with start_s = segments_df['end_s'].iloc[i] and end_s = segments_df['duration'].iloc[i]
#         new_row = {'utt_id': segments_df['utt_id'].iloc[i], 'start_s': segments_df['end_s'].iloc[i], 'end_s': segments_df['duration'].iloc[i], 'duration': segments_df['duration'].iloc[i], 'label': 'nonspeech'}
#         segments_df = segments_df.append(new_row, ignore_index=True)
# segments_df.head(5)

In [7]:
# #sort by utt_id, start_s, end_s
# segments_df = segments_df.sort_values(by=['utt_id', 'start_s'])
# #if it has same utt_id, new column 'segment_id' start from 1,2,3...
# segments_df['segment_id'] = segments_df.groupby('utt_id').cumcount()
# #add new column 'utt_id_segment_id' at first column and fill with utt_id + '_' + segment_id
# segments_df['utt_id_segment_id'] = segments_df['utt_id'].map(str) + '_' + segments_df['segment_id'].map(str)
# #move utt_id_segment_id to first column
# segments_df = segments_df[['utt_id_segment_id', 'utt_id', 'start_s', 'end_s', 'duration', 'label']]

In [ ]:

# segments_df[['utt_id_segment_id', 'utt_id', 'start_s', 'end_s']].to_csv('segments', index=False, sep=' ', header=None)
# segments_df[['utt_id_segment_id', 'label']].to_csv('utt2spk', index=False, sep=' ', header=None)

In [7]:
content = pd.read_csv('metadata/raw_multiskill_10per-val.csv', sep=',')
print(len(content))
#filter unique rows
content = content.drop_duplicates()
print(len(content))
content.to_csv('metadata/raw_multiskill_10per_val.csv', index=False, sep=',')

6173
3266


In [9]:
from pydub import AudioSegment

segments_out = pd.read_csv('data_test/segments_gr', sep=' ', header=None)
segments_out.columns=['utt_id_segment_id', 'utt_id', 'start_s', 'end_s']
path_df = pd.read_csv('data_test/wav.scp', sep=' ', header=None)
path_df.columns = ['utt_id', 'path']

for row in path_df.iterrows():
    audio_id = row[1]['utt_id']
    path = row[1]['path']
    list_segment = segments_out[segments_out['utt_id'] == audio_id]
    audio = AudioSegment.from_wav(path)
    for row_segment in list_segment.iterrows():
        print(row_segment[1]['utt_id_segment_id'])
        start_s = row_segment[1]['start_s']
        end_s = row_segment[1]['end_s']
        segment = audio[start_s*1000:end_s*1000]
        segment.export('path.../'+row_segment[1]['utt_id_segment_id']+'.wav', format='wav')
    




A1203478291#17758#9005549740015939_0


In [2]:
import requests

#https://users-dev.jenkins-x-viettel.iviet.com/v1/util/wav/A114966462%236203%239005547063353169

response = requests.get('https://users-dev.jenkins-x-viettel.iviet.com/v1/util/wav/A114966462%236203%239005547063353169')
print(response.reason)

OK
